This notebook is based on that written by Chevalier and can be found on [GitHub](https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition). The differences introduce in this notebook is the use of the Keras RNN library which is more modern compared to the LSTM RNN code used in the aforementioned basis for this notebook. 

# Setup

In [14]:
# Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics

import os

In [15]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
] 

# Extract dataset path

In [16]:
DATA_PATH = "data/"
DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)


Dataset is now located at: data/UCI HAR Dataset/


# Dataset Preparation

In [17]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Substract 1 to each output class for friendly 0-based indexing
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)


In [26]:
# Input Data

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training

learning_rate = 0.0025
lambda_loss_amount = 0.0015
epochs = 700
# epochs = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 1500


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 9) (2947, 1) 0.09913992 0.39567086
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [27]:
print(y_train)

[[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]


## Define RNN

In [41]:
model = keras.Sequential()
# 128 timesteps with 9 features each 
model.add(keras.Input(shape=(128,9)))
model.add(layers.SimpleRNN(n_hidden, activation='relu', return_sequences=True))
model.add(layers.SimpleRNN(n_hidden, activation='relu'))
model.add(layers.Dense(n_classes))
model.summary()

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate=learning_rate)
metrics = ["accuracy", "categorical_accuracy"]
model.compile(loss=loss, optimizer=optim, metrics=metrics)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_7 (SimpleRNN)     (None, 128, 32)           1344      
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 198       
Total params: 3,622
Trainable params: 3,622
Non-trainable params: 0
_________________________________________________________________


## Train the net

In [43]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/700
5/5 - 1s - loss: 0.7976 - accuracy: 0.6028 - categorical_accuracy: 0.1038
Epoch 2/700
5/5 - 1s - loss: 0.7665 - accuracy: 0.6328 - categorical_accuracy: 0.2499
Epoch 3/700
5/5 - 1s - loss: 0.7648 - accuracy: 0.6239 - categorical_accuracy: 0.2053
Epoch 4/700
5/5 - 2s - loss: 1.2537 - accuracy: 0.6163 - categorical_accuracy: 0.2035
Epoch 5/700
5/5 - 2s - loss: 2.3830 - accuracy: 0.4584 - categorical_accuracy: 0.1310
Epoch 6/700
5/5 - 2s - loss: 1.3956 - accuracy: 0.5065 - categorical_accuracy: 0.1266
Epoch 7/700
5/5 - 2s - loss: 1.1446 - accuracy: 0.5147 - categorical_accuracy: 0.1703
Epoch 8/700
5/5 - 1s - loss: 1.0637 - accuracy: 0.5739 - categorical_accuracy: 0.1651
Epoch 9/700
5/5 - 1s - loss: 1.0307 - accuracy: 0.5792 - categorical_accuracy: 0.1488
Epoch 10/700
5/5 - 1s - loss: 0.9967 - accuracy: 0.5747 - categorical_accuracy: 0.1182
Epoch 11/700
5/5 - 1s - loss: 0.9530 - accuracy: 0.6009 - categorical_accuracy: 0.0865
Epoch 12/700
5/5 - 1s - loss: 0.8938 - accuracy: 0.6

Epoch 96/700
5/5 - 2s - loss: 0.1747 - accuracy: 0.9331 - categorical_accuracy: 0.1813
Epoch 97/700
5/5 - 2s - loss: 0.1811 - accuracy: 0.9317 - categorical_accuracy: 0.1659
Epoch 98/700
5/5 - 1s - loss: 0.1735 - accuracy: 0.9357 - categorical_accuracy: 0.1789
Epoch 99/700
5/5 - 2s - loss: 0.1682 - accuracy: 0.9348 - categorical_accuracy: 0.1740
Epoch 100/700
5/5 - 2s - loss: 0.1507 - accuracy: 0.9397 - categorical_accuracy: 0.1763
Epoch 101/700
5/5 - 2s - loss: 0.1453 - accuracy: 0.9426 - categorical_accuracy: 0.1719
Epoch 102/700
5/5 - 2s - loss: 0.1476 - accuracy: 0.9421 - categorical_accuracy: 0.1736
Epoch 103/700
5/5 - 2s - loss: 0.1453 - accuracy: 0.9414 - categorical_accuracy: 0.1741
Epoch 104/700
5/5 - 2s - loss: 0.1492 - accuracy: 0.9437 - categorical_accuracy: 0.1706
Epoch 105/700
5/5 - 2s - loss: 0.1602 - accuracy: 0.9395 - categorical_accuracy: 0.1760
Epoch 106/700
5/5 - 2s - loss: 0.1615 - accuracy: 0.9396 - categorical_accuracy: 0.1727
Epoch 107/700
5/5 - 2s - loss: 0.151

5/5 - 1s - loss: 0.1274 - accuracy: 0.9415 - categorical_accuracy: 0.1695
Epoch 190/700
5/5 - 1s - loss: 0.1261 - accuracy: 0.9412 - categorical_accuracy: 0.1688
Epoch 191/700
5/5 - 1s - loss: 0.1251 - accuracy: 0.9467 - categorical_accuracy: 0.1689
Epoch 192/700
5/5 - 1s - loss: 0.1279 - accuracy: 0.9418 - categorical_accuracy: 0.1688
Epoch 193/700
5/5 - 2s - loss: 0.1300 - accuracy: 0.9450 - categorical_accuracy: 0.1687
Epoch 194/700
5/5 - 2s - loss: 0.1276 - accuracy: 0.9444 - categorical_accuracy: 0.1687
Epoch 195/700
5/5 - 1s - loss: 0.1261 - accuracy: 0.9425 - categorical_accuracy: 0.1688
Epoch 196/700
5/5 - 1s - loss: 0.1301 - accuracy: 0.9464 - categorical_accuracy: 0.1692
Epoch 197/700
5/5 - 1s - loss: 0.1263 - accuracy: 0.9514 - categorical_accuracy: 0.1689
Epoch 198/700
5/5 - 1s - loss: 0.1268 - accuracy: 0.9442 - categorical_accuracy: 0.1687
Epoch 199/700
5/5 - 1s - loss: 0.1279 - accuracy: 0.9476 - categorical_accuracy: 0.1681
Epoch 200/700
5/5 - 1s - loss: 0.1239 - accura

Epoch 283/700
5/5 - 2s - loss: 0.1408 - accuracy: 0.9486 - categorical_accuracy: 0.1668
Epoch 284/700
5/5 - 2s - loss: 0.1370 - accuracy: 0.9491 - categorical_accuracy: 0.1692
Epoch 285/700
5/5 - 2s - loss: 0.1350 - accuracy: 0.9452 - categorical_accuracy: 0.1688
Epoch 286/700
5/5 - 2s - loss: 0.1279 - accuracy: 0.9506 - categorical_accuracy: 0.1685
Epoch 287/700
5/5 - 2s - loss: 0.1263 - accuracy: 0.9450 - categorical_accuracy: 0.1685
Epoch 288/700
5/5 - 2s - loss: 0.3021 - accuracy: 0.8909 - categorical_accuracy: 0.2078
Epoch 289/700
5/5 - 2s - loss: 0.4058 - accuracy: 0.8311 - categorical_accuracy: 0.2205
Epoch 290/700
5/5 - 2s - loss: 0.4037 - accuracy: 0.8300 - categorical_accuracy: 0.1340
Epoch 291/700
5/5 - 2s - loss: 0.3823 - accuracy: 0.8372 - categorical_accuracy: 0.1537
Epoch 292/700
5/5 - 2s - loss: 0.3329 - accuracy: 0.8511 - categorical_accuracy: 0.2137
Epoch 293/700
5/5 - 2s - loss: 0.3148 - accuracy: 0.8553 - categorical_accuracy: 0.2414
Epoch 294/700
5/5 - 2s - loss: 0

5/5 - 2s - loss: 0.1265 - accuracy: 0.9499 - categorical_accuracy: 0.1663
Epoch 377/700
5/5 - 2s - loss: 0.1243 - accuracy: 0.9508 - categorical_accuracy: 0.1669
Epoch 378/700
5/5 - 2s - loss: 0.1239 - accuracy: 0.9499 - categorical_accuracy: 0.1663
Epoch 379/700
5/5 - 2s - loss: 0.1287 - accuracy: 0.9468 - categorical_accuracy: 0.1687
Epoch 380/700
5/5 - 2s - loss: 0.1234 - accuracy: 0.9472 - categorical_accuracy: 0.1673
Epoch 381/700
5/5 - 2s - loss: 0.1236 - accuracy: 0.9494 - categorical_accuracy: 0.1665
Epoch 382/700
5/5 - 2s - loss: 0.1213 - accuracy: 0.9521 - categorical_accuracy: 0.1677
Epoch 383/700
5/5 - 2s - loss: 0.1210 - accuracy: 0.9487 - categorical_accuracy: 0.1674
Epoch 384/700
5/5 - 2s - loss: 0.1271 - accuracy: 0.9487 - categorical_accuracy: 0.1649
Epoch 385/700
5/5 - 2s - loss: 0.1235 - accuracy: 0.9499 - categorical_accuracy: 0.1680
Epoch 386/700
5/5 - 2s - loss: 0.1210 - accuracy: 0.9490 - categorical_accuracy: 0.1670
Epoch 387/700
5/5 - 2s - loss: 0.1204 - accura

Epoch 470/700
5/5 - 2s - loss: 0.1010 - accuracy: 0.9543 - categorical_accuracy: 0.1669
Epoch 471/700
5/5 - 2s - loss: 0.1030 - accuracy: 0.9557 - categorical_accuracy: 0.1666
Epoch 472/700
5/5 - 2s - loss: 0.1065 - accuracy: 0.9523 - categorical_accuracy: 0.1677
Epoch 473/700
5/5 - 2s - loss: 0.1017 - accuracy: 0.9542 - categorical_accuracy: 0.1666
Epoch 474/700
5/5 - 2s - loss: 0.1028 - accuracy: 0.9551 - categorical_accuracy: 0.1659
Epoch 475/700
5/5 - 2s - loss: 0.1020 - accuracy: 0.9550 - categorical_accuracy: 0.1672
Epoch 476/700
5/5 - 2s - loss: 0.1013 - accuracy: 0.9577 - categorical_accuracy: 0.1666
Epoch 477/700
5/5 - 2s - loss: 0.1021 - accuracy: 0.9561 - categorical_accuracy: 0.1668
Epoch 478/700
5/5 - 2s - loss: 0.0997 - accuracy: 0.9547 - categorical_accuracy: 0.1666
Epoch 479/700
5/5 - 2s - loss: 0.0998 - accuracy: 0.9553 - categorical_accuracy: 0.1670
Epoch 480/700
5/5 - 2s - loss: 0.0998 - accuracy: 0.9562 - categorical_accuracy: 0.1665
Epoch 481/700
5/5 - 2s - loss: 0

5/5 - 2s - loss: 0.0889 - accuracy: 0.9615 - categorical_accuracy: 0.1668
Epoch 564/700
5/5 - 2s - loss: 0.0892 - accuracy: 0.9621 - categorical_accuracy: 0.1668
Epoch 565/700
5/5 - 2s - loss: 0.0907 - accuracy: 0.9599 - categorical_accuracy: 0.1666
Epoch 566/700
5/5 - 2s - loss: 0.0912 - accuracy: 0.9597 - categorical_accuracy: 0.1666
Epoch 567/700
5/5 - 2s - loss: 0.0915 - accuracy: 0.9611 - categorical_accuracy: 0.1668
Epoch 568/700
5/5 - 2s - loss: 0.0901 - accuracy: 0.9606 - categorical_accuracy: 0.1669
Epoch 569/700
5/5 - 2s - loss: 0.0909 - accuracy: 0.9610 - categorical_accuracy: 0.1666
Epoch 570/700
5/5 - 2s - loss: 0.0912 - accuracy: 0.9585 - categorical_accuracy: 0.1669
Epoch 571/700
5/5 - 2s - loss: 0.0938 - accuracy: 0.9591 - categorical_accuracy: 0.1673
Epoch 572/700
5/5 - 2s - loss: 0.0995 - accuracy: 0.9521 - categorical_accuracy: 0.1663
Epoch 573/700
5/5 - 2s - loss: 0.1074 - accuracy: 0.9532 - categorical_accuracy: 0.1668
Epoch 574/700
5/5 - 2s - loss: 0.1065 - accura

Epoch 657/700
5/5 - 2s - loss: 0.1169 - accuracy: 0.9457 - categorical_accuracy: 0.1665
Epoch 658/700
5/5 - 2s - loss: 0.1103 - accuracy: 0.9540 - categorical_accuracy: 0.1676
Epoch 659/700
5/5 - 2s - loss: 0.1111 - accuracy: 0.9529 - categorical_accuracy: 0.1654
Epoch 660/700
5/5 - 2s - loss: 0.1075 - accuracy: 0.9527 - categorical_accuracy: 0.1673
Epoch 661/700
5/5 - 2s - loss: 0.1082 - accuracy: 0.9569 - categorical_accuracy: 0.1661
Epoch 662/700
5/5 - 2s - loss: 0.0991 - accuracy: 0.9574 - categorical_accuracy: 0.1658
Epoch 663/700
5/5 - 2s - loss: 0.0968 - accuracy: 0.9577 - categorical_accuracy: 0.1683
Epoch 664/700
5/5 - 2s - loss: 0.0948 - accuracy: 0.9612 - categorical_accuracy: 0.1661
Epoch 665/700
5/5 - 2s - loss: 0.0944 - accuracy: 0.9584 - categorical_accuracy: 0.1668
Epoch 666/700
5/5 - 2s - loss: 0.0914 - accuracy: 0.9616 - categorical_accuracy: 0.1666
Epoch 667/700
5/5 - 2s - loss: 0.0893 - accuracy: 0.9614 - categorical_accuracy: 0.1665
Epoch 668/700
5/5 - 2s - loss: 0

In [44]:
model.evaluate(X_test, y_test, batch_size=batch_size, verbose=2)

2/2 - 1s - loss: 0.9780 - accuracy: 0.8544 - categorical_accuracy: 0.1690


[0.9779502749443054, 0.854428231716156, 0.16898541152477264]

In [46]:
model.save("HAR-RNN-Classifier")

INFO:tensorflow:Assets written to: HAR-RNN-Classifier\assets
